## 1. Importowanie bibliotek

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import sys
!conda install --yes --prefix {sys.prefix} watermark
from numpy import unique
from numpy import mean
from urllib.request import urlopen
import warnings
warnings.filterwarnings('ignore')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



## 2. Wczytanie danych

In [2]:
url = 'https://github.com/kcisowska/cleaning_titanic/blob/main/TitanicMess.tsv?raw=true '

In [3]:
dataset = pd.read_csv(url, sep='\t', decimal = ",")

In [4]:
dataset.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic


## 3. Informacje o zbiorze

Początkowa ilość wierszy i kolumn:

In [5]:
dataset.shape

(892, 13)

Kolumny z zbiorze

In [6]:
dataset.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'ship'],
      dtype='object')

Ilośc brakujących wartości w zbiorze

In [7]:
dataset.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          685
Embarked         2
ship             0
dtype: int64

Definujemy różne wartości NA w zbiorze, ponieważ python uznaje tylko wartość NaN jako brakującą.

In [8]:
missing_values = ["n/a", "na", "--", "NaN", "nan", " "]

In [9]:
ds = pd.read_csv(url,sep='\t', decimal = ",", na_values = missing_values)

Wszystkie brakujące wartości w zbiorze okazały się być zdefiniowane jako NaN:

In [10]:
ds.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          685
Embarked         2
ship             0
dtype: int64

## 4. Usuwanie duplikatów w zbiorze

Tworzenie zmiennej dla duplikatów w zbiorze

In [11]:
dups = ds.duplicated()

Weryfikowanie czy duplikaty istnieją

In [12]:
print(dups.any())

True


Wypisanie duplikatów ze zbioru

In [13]:
print(ds[dups])

     PassengerId  Survived  Pclass                                Name  \
13            11         1       3  Sandstrom, Miss. Marguerite Ru&5$$   
23            11         1       3  Sandstrom, Miss. Marguerite Ru&5$$   
520          225         1       1        Hoyt, Mr. Frederick Maxfield   

        Sex Age  SibSp  Parch   Ticket  Fare Cabin Embarked     ship  
13   female   4      1      1  PP 9549  16,7    G6        S  Titanic  
23   female   4      1      1  PP 9549  16,7    G6        S  Titanic  
520    male  38      1      0    19943    90   C93        S  Titanic  


Usunięcie duplikatów

In [14]:
ds.drop_duplicates(inplace=True)

Wypisanie ilości wierszy w zbiorze(widzimy, że liczba zmniejszyła się o 3):

In [15]:
print(ds.shape)

(889, 13)


Ponowna weryfikacja czy w zbiorze nie pozostały duplikaty:

In [16]:
dups = ds.duplicated()

In [17]:
print(dups.any())

False


## 5. Usuwanie kolumn z pojedynczą wartością unikalną

Sprawdzanie czy istnieją kolumny posiadające tylko jedną wartość unikalną:

In [18]:
counts = ds.nunique()

In [19]:
print(counts)

PassengerId    888
Survived         2
Pclass           3
Name           889
Sex              6
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           250
Cabin          145
Embarked         6
ship             1
dtype: int64


Tylko kolumna ship ma pojedynczą wartość i powinna zostać usunięta

Identyfikator kolumny do usunięcia

In [20]:
to_del = [i for i,v in enumerate(counts) if v == 1]
print(to_del)

[12]


Usuwamy kolumnę z pojedynczą wartością:

In [21]:
ds.drop('ship', axis=1, inplace=True)

Kolumna ship została usunięta:

In [22]:
print(ds.shape)

(889, 12)


In [23]:
print(ds.nunique())

PassengerId    888
Survived         2
Pclass           3
Name           889
Sex              6
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           250
Cabin          145
Embarked         6
dtype: int64


## 6. Usuwanie kolumn, których procent wartości brakujących przekracza 75 %

Wyświetl procent brakujących danych w kolumnach:

In [24]:
percent_missing = ds.isnull().sum() * 100 / len(ds)

In [25]:
print(percent_missing)

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.460067
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.052868
Embarked        0.224972
dtype: float64


Usuń kolumny, których liczba brakujących danych jest większa niż 75 %

In [26]:
perc = 75.0
min_count =  int(((100-perc)/100)*ds.shape[0] + 1)
mod_ds = ds.dropna( axis=1, 
                thresh=min_count)

In [27]:
print(mod_ds)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   
891         1000         1       1   

                                                  Name     Sex  Age  SibSp  \
0                              Braund, Mr. Owen Harris    male   22      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female   38      1   
2                               Heikkinen, Miss. Laina  female   26      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   35      1   
4                             Allen, Mr. William Henry    male   35      0   
..                                                 ...     ...  ...    ... 

Kolumna Cabin została usunięta

## 7. Zastępowanie brakujących wartości w kolumnach nie numerycznych

Zastępujemy brakujące wartości w kolumnie Embarked

In [28]:
mod_ds.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [29]:
mod_ds["Embarked"].describe()

count     887
unique      6
top         S
freq      640
Name: Embarked, dtype: object

Najczęsciej występująca wartością w Embarked jest S dlatego za brakujące wartości w tej kolumnie wstawimy S

In [30]:
mod_ds["Embarked"].fillna('S', inplace=True)

In [31]:
mod_ds.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64

## 8. Usuwanie wartości odstających

### Przygotowanie danych w kolumnie Age przed wyliczeniem kwantyli

Sortowanie

In [32]:
sorted(mod_ds['PassengerId'])

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 1

Na początku musimy zanleźć wszystkie liczby używające jako separator dziesiętny przecinek:

In [33]:
mod_ds[mod_ds['Age'].str.contains(',', na = False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
57,58,0,3,"Novel, Mr. Mansouer",male,"28,5",0,0,2697,"7,2292",C
78,79,1,2,"Caldwell, Master. Alden Gates",male,"0,83",0,2,248738,29,S
111,112,0,3,"Zabour, Miss. Hileni",female,"14,5",1,0,2665,"14,4542",C
116,117,0,3,"Connors, Mr. Patrick",male,"70,5",0,0,370369,"7,75",Q
122,123,0,2,"Nasser, Mr. Nicholas",male,"32,5",1,0,237736,"30,0708",C
123,124,1,2,"Webber, Miss. Susan",female,"32,5",0,0,27267,13,S
148,149,0,2,"Navratil, Mr. Michel (""Louis M Hoffman"")",male,"36,5",0,2,230080,26,S
152,153,0,3,"Meo, Mr. Alfonzo",male,"55,5",0,0,A.5. 11206,"8,05",S
153,154,0,3,"van Billiard, Mr. Austin Blyler",male,"40,5",0,2,A/5. 851,"14,5",S
203,204,0,3,"Youseff, Mr. Gerious",male,"45,5",0,0,2628,"7,225",C


I zastąpić w tych liczbach "," kropką, ponieważ inaczej liczba zostanie zastąpiona wartością brakującą, ponieważ nie będzie rozpoznawana jako liczba

In [34]:
mod_ds['Age'] = mod_ds['Age'].str.replace(",", ".")

In [35]:
mod_ds[mod_ds['Age'].str.contains(',', na = False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


Następnie zanim wyliczymy kwantyle. Musimy przetransformować nienumeryczne wartości w kolumnie Age na wartości NaN, dzięki czemu będziemy mogli wyliczyć kwantyle

In [36]:
mod_ds["Age"] = pd.to_numeric(mod_ds["Age"], errors='coerce')

Następnie zanim usuniemy wartości odstajace nadamy wartością brakującym wartość, która nie znajduje się w usuwanym zakresie, aby dane te nie zostały przypadkowo usunięte

In [37]:
mod_ds.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64

In [38]:
mod_ds["Age"].fillna(1.7, inplace=True)

### Wyliczanie kwantyli oraz usuwanie wartości poza ich zakresem

Ustalamy wartość kwantyli dla wieku pasażerów zakładając, że najmłodszy pasażer nie miał mniej niż 5 miesiący, a najstarszy pasażer nie miał więcej niż 80 lat

In [39]:
q_low = mod_ds["Age"].quantile(0.00224)
q_hi  = mod_ds["Age"].quantile(0.99775)

print(q_low,q_hi)

0.3827903999999992 80.34000000001129


In [40]:
df_filtered = mod_ds[(mod_ds["Age"] < q_hi) & (mod_ds["Age"] > q_low)]

In [41]:
print(df_filtered)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   
891         1000         1       1   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

Wartości odstające zostały usunięte, więc następnie przywrócimy wartości NaN

In [42]:
mapping = {1.7: ' '}

In [43]:
df_filtered = df_filtered.replace({'Age': mapping})

In [44]:
df_filtered["Age"] = pd.to_numeric(df_filtered["Age"], errors='coerce')

In [45]:
df_filtered.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64

Ponownie liczba wartości brakujących w kolumnie Age wynosi 173

### Wartości odstające dla kolumny PassengerId:

In [46]:
df_filtered["PassengerId"] = pd.to_numeric(df_filtered["PassengerId"], errors='coerce')

In [47]:
q_hi2  = df_filtered["PassengerId"].quantile(0.99888)

print(q_hi2)

892.0812800000086


In [48]:
df_filtered = df_filtered[(df_filtered["PassengerId"] < q_hi2)]

In [49]:
print(df_filtered)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

Pozbyliśmy się ostaniego wiersza z PassengerId równym 1000, ponieważ ostanią prawidłową, najwyższą wartością powinno być 891

### Wartości odstające dla kolumny Fare:

In [50]:
df_filtered[df_filtered['Fare'].str.contains('-', na = False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
245,246,0,1,"Minahan, Dr. William Edward",male,44.0,2,0,19928,-90,Q


In [51]:
ds = df_filtered.drop([245])

In [52]:
ds[ds['Fare'].str.contains('-', na = False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [53]:
print(ds)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

Usunięto wartości ujemne, ponieważ Fare nie może być mniejsza niż 0.

## 9. Ujednolicenie danych oraz usunięcie literówek

### W kolumnie Name usuwamy wiersze z symbolami specjalnycmi np $ czy &

Wyszukjemy wiersze ze znakami specjalnymi

In [54]:
ds[ds['Name'].str.contains('&')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
10,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4.0,1,1,PP 9549,"16,7",S


Zastępujemy wartość niepotrzebną &5 prawidołowym końcem wyrazu:

In [55]:
ds['Name'] = ds['Name'].str.replace("&5", "t")

Sprawdzamy czy kolumna nie zawiera już wierszy ze znakiem &

In [56]:
ds[ds['Name'].str.contains('&')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


Następnie ponownie wyświetlamy linię, ponieważ zawierała ona jeszcze znak $, niestety znaków spoza ASCI nie można wyszukać przy użyciu contains.

In [57]:
print(ds.loc[[10]])

    PassengerId  Survived  Pclass                               Name     Sex  \
10           11         1       3  Sandstrom, Miss. Marguerite Rut$$  female   

    Age  SibSp  Parch   Ticket  Fare Embarked  
10  4.0      1      1  PP 9549  16,7        S  


Usuwamy z kolumny Name wszystkie znaki spoza ASCI w tym $

In [58]:
ds['Name'] = ds['Name'].str.replace(r'[^\x00-\x7f]', '')

In [59]:
ds['Name'] = ds['Name'].str.replace(r'[$$]', '')

Sprawdzamy czy linia nie zawiera już znaków specjalnych takich jak $$ i &

In [60]:
print(ds.loc[[10]])

    PassengerId  Survived  Pclass                             Name     Sex  \
10           11         1       3  Sandstrom, Miss. Marguerite Rut  female   

    Age  SibSp  Parch   Ticket  Fare Embarked  
10  4.0      1      1  PP 9549  16,7        S  


Pozbyliśmy się z tej kolumny zarówno znaków specjalnych jak i niepotrzebnych znaków

### Następnie w kolumnie Embarked ujednolicamy nazwy portów do skrótów jednoliterowych

In [61]:
ds[ds["Embarked"] == "So"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13,So
221,222,0,2,"Bracken, Mr. James H",male,27.0,0,0,220367,13,So


In [62]:
ds['Embarked'] = ds['Embarked'].str.replace("So", "S")

In [63]:
ds[ds["Embarked"] == "So"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [64]:
ds[ds["Embarked"] == "Qe"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
626,627,0,2,"Kirkland, Rev. Charles Leonard",male,57.0,0,0,219533,"12,35",Qe


In [65]:
ds['Embarked'] = ds['Embarked'].str.replace("Qe", "Q")

In [66]:
ds[ds["Embarked"] == "Qe"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [67]:
ds[ds["Embarked"] == "Co"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
598,599,0,3,"Boulos, Mr. Hanna",male,NaN,0,0,2664,"7,225",Co


In [68]:
ds['Embarked'] = ds['Embarked'].str.replace("Co", "C")

In [69]:
ds[ds["Embarked"] == "Co"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [70]:
print(ds.nunique())

PassengerId    882
Survived         2
Pclass           3
Name           883
Sex              5
Age             89
SibSp            7
Parch            6
Ticket         676
Fare           249
Embarked         3
dtype: int64


### Weryfikujemy pozostałe w zbiorze wartości unikalne

Zauważmy, że mamy tylko 882 unikalnych wartości dla PassengerId mimo, że posiadamy 883 rekordy, oznacza to, że jedno id nie jest unikalne i należy zweryfikować, czy nie jest to duplikat.

In [71]:
dups = ds["PassengerId"].duplicated()

In [72]:
print(ds["PassengerId"][dups])

678    225
Name: PassengerId, dtype: int64


Tak okazało się, że mamy 1 duplikat id wypiszmy teraz oba te wiersze, żeby zweryfikować czy posiadają te same dane:

In [73]:
print(ds.loc[[224,678]])

     PassengerId  Survived  Pclass                           Name   Sex   Age  \
224          225         1       1   Hoyt, Mr. Frederick Maxfield  male  38.0   
678          225         1       1  Hoytt, Mr. Frederick Maxfield  male  38.0   

     SibSp  Parch Ticket Fare Embarked  
224      1      0  19943   90        S  
678      1      0  19943   90        S  


Zauważmy, że sa te te same dane jednak wcześniej podczas wykrywania duplikatów rekord ten nie wyświetlił się, ponieważ dane różnią się 1 literą. Teraz usuniemy zduplikowaną linię zawierjącą literówkę z podwojoną literą.

In [74]:
ds = ds.drop([678])

Weryfikujemy czy nie pozostały żadne duplikaty w PassengerId

In [75]:
dups = ds["PassengerId"].duplicated()

## 10. Uzupełniamy pozostałe wartości brakujące

Sprawdzamy czy typ danych dla kolumny Age umożliwia wyliczenie średniej oraz czy wartości brakujące są wciąż prawidłowo czytane

In [76]:
ds["Age"] = pd.to_numeric(ds["Age"], errors='coerce')

In [77]:
print (ds.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare            object
Embarked        object
dtype: object


In [78]:
print(ds)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [79]:
ds.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64

Wyliczamy średnią dla Age i zastępujemy nią wartości brakujace

In [80]:
mean_age = ds["Age"].mean()

In [81]:
mean_age = round(mean_age, 0)

In [82]:
print(mean_age)

30.0


In [83]:
ds["Age"].fillna(mean_age, inplace=True)

In [84]:
print(ds)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

Widzimy, że w wierszu 888 gdzie poprzenio była wartość NaN jest obecnie wartość 30

In [85]:
ds.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

Widzimy, że obecnie pozbyliśmy się wszystkich wartośći brakujących ze zbioru, więc zapiszemy zbiór do pliku.

In [86]:
ds.to_csv('TitanicCleaned.tsv', header= True, sep='\t', decimal = ",")

### Tworzymy zależności dla pliku reguirements.txt

In [87]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p pandas,numpy,matplotlib,seaborn,watermark 

# date
print (" ")
%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.7.10
IPython version      : 7.20.0

pandas    : 1.1.3
numpy     : 1.19.2
matplotlib: 3.3.2
seaborn   : 0.11.0
watermark : 2.2.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 4.19.150+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

 
Last updated: Sun May 09 2021 12:31:51UTC

